# The COMPAS class

## 1. Introduction

In order to calculate the formation rate of a type of double compact object at a given redshift, we need to know:

1. The grid of metallicities we assume for the integral

2. The amount of solar mass evolved per metallicity per system $\frac{d}{dM_\odot}(Z)$

3. The type of double compact object (DCO) we are interested in 

4. The metallicity $Z$ at which each DCO formed and the delay time (time from formation till merger) $t_c$ for each DCO


Given a time at which the DCO merges, we can then calculate the time at which it formed, so we can recover the MSSFR ($\frac{dM_\odot}{dt}$).


In addition, we need to know the component masses of the system in order to calculate any selection effects.

The COMPAS class (ClassCOMPAS.py) stores the information that we need, and provides quick access when calculating the rates.

### 1.1 Paths

In [7]:
import os
pathNoteBook     = os.getcwd()
pathClassCOMPAS  = pathNoteBook + '/PythonScripts/'
pathData         = '/path-to-data/'

### 1.2 Imports

In [8]:
import numpy as np
import sys
sys.path.append(pathClassCOMPAS)
import ClassCOMPAS

## 2. Creating an instance of the COMPAS class

To create an instance of the COMPAS output class, we need to specify the following attributes:

    path           = '../'
    fileName       = 'COMPAS_Output.h5'
    
    lazyData       = True
    
    Mlower         = None
    Mupper         = None
    
    binaryFraction = None

`path`

Path to the h5-data. 

`fileName`

Name of the h5 data

`lazyData`

Indicates we store additional info like the mass ratios, chirpmasses of each DCO system. In principle this could be done externally, but this is slightly easier when plotting/combining info later on (but it does use more memory).
    
    
`Mlower`

Lower limit used for $M_1$ in the pythonSubmit of the simulation. Required to recover `true' amount of $M_{\odot}$ evolved (see step 4)

`Mupper`

Upper limit used for $M_1$ in the pythonSubmit of the simulation. Required to recover `true' amount of $M_{\odot}$ evolved (see step 4)

`binaryFraction`


Assumed fraction of stars in binaries. Required to recover `true' amount of $M_{\odot}$ evolved (see step 4)


In [9]:
#I assume all the defaults and just set the path
COMPASData = ClassCOMPAS.COMPASData(path=pathData)


ClassCOMPAS: Remember to self.setCOMPASDCOmask()
                    then self.setCOMPASData()
          and optionally self.setGridAndMassEvolved() if using a metallicity grid


The output are reminders which will be explained in next steps.

## 3. Total mass evolved

In a COMPAS simulation we often only evolve massive stars. This means that the total mass in our simulation does not represent the total mass evolved in all stars. Here we recover an estimate of what that total mass is using the lower and upper mass for the primary from your program options (via the command line or a python submit file), and assuming a binary fraction.

The code will then check your data, and test per metallicity how much mass is evolved. It assumes the metallicities are subject to the same program options, but maybe due to sampling had a different number of systems. It also recovers from the data what metallicity grid is used.

In [10]:
COMPASData.Mlower = 15
COMPASData.Mupper = 150
COMPASData.binaryFraction =0.7

COMPASData.setGridAndMassEvolved()

## 3.1 The grid of metallicities we assume for the integral

By default, the COMPAS class will automatically try to recover the metallicity grid from the data. It assumes that metallicities of all the systems in the h5-data represent the assumed metallicity grid for the calculation.

    metallicities        = Data['BSE_System_Parameters']['Metallicity@ZAMS(1)'][()]
    self.metallicityGrid = np.unique(metallicities)

In principle you could instead overwrite this with your own metallicity grid: remember to reassign the metallicities of each DCO and the amount of solar mass evolved per metallicity.

However, we leave it at that for now. You can access the grid by printing:

In [11]:
print(COMPASData.metallicityGrid)


[0.00142]


## 4. The amount of solar mass evolved per system per Z

Again, by default, the COMPAS class will automatically recover the amount of 'true' solar mass evolved per system using the `totalMassEvolvedPerZ` script, and by reading the total mass per system in the simulation. This recovers an amount of solar mass per metallicity of the metallicity grid (units $M_{\odot}$).

In [12]:
print(COMPASData.totalMassEvolvedPerZ)


[423202.1360283]


## 5. Selecting the DCO type of interest

We use a boolean mask to recover the metallicities, delay times, and other parameters, of the DCOs of interest. The boolean mask, which has the same length as the DCO h5 group, selects the systems we want to include in the calculation. 

You could set your own mask using any combination you want by:

    maskDCO = some criteria you like on the h5 data
    COMPASData.DCOmask = maskDCO
    

However, we are usually interested in a specific group of merging DCOs, assuming a type of physics. The `setCOMPASDCOmask()` function allows you to quickly set the mask without doing the slicing yourself, and takes the following arguments:

`types`

    Type of double compact object (DCO) to mask for.
    One of: {'BBH', 'BNS', 'BHNS', 'CHE_BBH', 'NON_CHE_BBH', 'All'}, default = 'BBH'

    'BBH' masks for all binary black holes.
    'CHE_BBH' masks for only those BBHs where both constituent stars evolved as CH stars for their entire MS lifetime.
    'NON_CHE_BBH' masks for only those BBHs where one or both constituent stars did not evolve as CH stars for their entire MS lifetime.
    'BNS' masks for all binary neutron stars.
    'BHNS' masks for all black hole - neutron star binaries.
    'All' masks for BBHs (both CHE and NON-CHE), BNSs, and BHNSs.

`withinHubbleTime`

    One of: {True, False}, default = True
    If True, only use DCOs that merge within a Hubble time.

`pessimistic`

    One of: {True, False}, default = True
    If True, mask out DCOs that have formed through a common-envelope event involving a Hertzsprung-gap donor.

`noRLOFafterCEE`

    One of: {True, False}, default = True
    If True, mask out DCOs that have at some point experienced RLOF immediately after a common-envelope event.

In [17]:
COMPASData.setCOMPASDCOmask(types='BBH', pessimistic=True)

In [18]:
#Check if we have any system meeting the criteria
print('nr systems =%s ' %(np.sum(COMPASData.DCOmask)))


nr systems =19 


## 6. Get the metallicities and delay times

Using the DCO mask defined in step 3, the class can now get the parameters of interest for each merging DCO:


In [19]:
COMPASData.setCOMPASData()

Now the data is set and you are ready to go.


## 7. For different data

If you have your own simulation which is different from the COMPAS data, or you want to test a toy model, you can still use the set of pipelines for the cosmic integration.

The only thing you need to do is construct your own arrays.

Create an instance of the clasCOMPAS without a path:

In [20]:
MockData = ClassCOMPAS.COMPASData(path=None)

Just to double check you create instance of ClassCOMPAS without path/Data
ClassCOMPAS: Remember to self.setCOMPASDCOmask()
                    then self.setCOMPASData()
          and optionally self.setGridAndMassEvolved() if using a metallicity grid


Then manually set each array for:

    #grid for integral
    MockData.metallicityGrid
    MockData.totalMassEvolvedPerZ #same length array as grid
    
    #Metallicity of each system corresponding to a grid-point
    MockData.metallicitySystems
    MockData.delayTimes  #Myr
    MockData.mass1       #Msun
    MockData.mass2       #Msun    
    #All four arrays are same length since they correspond to number of systems

All other pipelines just read arrays and are independent of the data.

So you could sample from the IMF for $M_1$ and $M_2$, create a grid in metallicities and uniformly sample from the grid, and then sample from a $t^{-1}$ delay time distribution (just note the assumed units :))

If you then set `totalMassEvolvedperZ` to an array of ones, you can at least create predictions for the shape of the merger rate distributions - since this array is only a normalization affecting the absolute rates.